In [63]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

In [64]:
# Initialize Pinecone
pincecone = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pincecone.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)



In [65]:
import json
# Load the review data
data = json.load(open("reviews.json"))

In [66]:
# Create embeddings for each review
for review in data["restaurant"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["restaurant"],
            "metadata":{
                "review": review["review"],
                "cuisine": review["cuisine"],
                "rating": review["rating"],
            }
        }
    )

In [67]:
processed_data[0]
client = OpenAI()

In [68]:

index = pc.Index("rag")

# Insert the embeddings into the Pinecone index
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

print("Upsert response:", upsert_response)


Upsert response: {'upserted_count': 40}


In [69]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}